# 05. NFL Season Simulation (2024–2025)

**Goal:**  
Simulate the 2024 NFL season using pre-game win probabilities from Notebook 04 and compute expected wins, Monte Carlo win distributions, and toy Super Bowl probabilities.

**Contents:**  
- Load `matchups_pre_game_2019_2024_with_clusters.csv`  
- Filter 2024 games and compute expected wins  
- Compare expected vs actual 2024 results (MAE, RMSE)  
- Monte Carlo simulation (default: 10,000 iterations)  
- Win distribution metrics:
  - mean_wins, std_wins  
  - probability of ≥10 wins, ≤5 wins  
- Softmax-based toy Super Bowl probability estimation  
- Identify over-/under-performing teams relative to expectation  

**Outputs:**  
- DataFrames: expected wins, Monte Carlo statistics, SB probabilities  
- Tables & visual summaries for use in GitHub portfolio

## 1. Imports & paths

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score,
    brier_score_loss,
    roc_auc_score,
    confusion_matrix,
)

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    HAS_XGB = False

# Paths
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent  # assuming /project/notebooks/model
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"

print("NOTEBOOK_DIR :", NOTEBOOK_DIR)
print("PROJECT_ROOT :", PROJECT_ROOT)
print("DATA_DIR     :", DATA_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)

NOTEBOOK_DIR : /Users/minseobeom/Desktop/nfl-epa-analysis/notebooks
PROJECT_ROOT : /Users/minseobeom/Desktop/nfl-epa-analysis
DATA_DIR     : /Users/minseobeom/Desktop/nfl-epa-analysis/data
PROCESSED_DIR: /Users/minseobeom/Desktop/nfl-epa-analysis/data/processed


##  2. Load pre-game matchup dataset

In [3]:
# 1. Try to locate the pre-game matchup CSV

candidate_files = [
    PROCESSED_DIR / "pre_game_matchups_2019_2024_with_clusters.csv",
    PROCESSED_DIR / "matchups_pre_game_2019_2024_with_clusters.csv",
    PROCESSED_DIR / "matchups_pre_game_2019_2024.csv",
    # If you saved it under notebooks/data/processed
    PROJECT_ROOT / "notebooks" / "data" / "processed" / "matchups_pre_game_2019_2024_with_clusters.csv",
]

matchups_path = None
print("Searching for matchup CSV in candidate paths:")
for p in candidate_files:
    exists = p.exists()
    print(f"  {p} -> {exists}")
    if exists and matchups_path is None:
        matchups_path = p

if matchups_path is None:
    raise FileNotFoundError(
        "Could not find the pre-game matchup CSV.\n"
        "Open NFL_PreGame_RollingEPA_Modeling.ipynb and save the final `matchups` "
        "DataFrame as a CSV into data/processed/, then update the filename here."
    )

print(f"\n[INFO] Using matchup CSV: {matchups_path}")
matchups = pd.read_csv(matchups_path)

print("matchups shape:", matchups.shape)
print(matchups.head(10))

Searching for matchup CSV in candidate paths:
  /Users/minseobeom/Desktop/nfl-epa-analysis/data/processed/pre_game_matchups_2019_2024_with_clusters.csv -> False
  /Users/minseobeom/Desktop/nfl-epa-analysis/data/processed/matchups_pre_game_2019_2024_with_clusters.csv -> True
  /Users/minseobeom/Desktop/nfl-epa-analysis/data/processed/matchups_pre_game_2019_2024.csv -> False
  /Users/minseobeom/Desktop/nfl-epa-analysis/notebooks/data/processed/matchups_pre_game_2019_2024_with_clusters.csv -> False

[INFO] Using matchup CSV: /Users/minseobeom/Desktop/nfl-epa-analysis/data/processed/matchups_pre_game_2019_2024_with_clusters.csv
matchups shape: (1675, 71)
   season  week          game_id home_gameday home_home_team home_away_team  \
0    2019     1  2019_01_DET_ARI   2019-09-08            ARI            DET   
1    2019     3  2019_03_CAR_ARI   2019-09-22            ARI            CAR   
2    2019     4  2019_04_SEA_ARI   2019-09-29            ARI            SEA   
3    2019     6  2019_06_

## 3. Define features & train/test split

In [4]:
# 2. Define feature columns

feature_cols_numeric = [
    "diff_rolling_off_epa_3",
    "diff_rolling_def_epa_3",
    "diff_rolling_net_epa_3",
    "diff_rest_days",
    "diff_prev_off_epa_mean",
    "diff_prev_def_epa_mean",
    "diff_prev_win_pct",
]

feature_cols_categorical = [
    "home_prev_cluster_name",
    "away_prev_cluster_name",
    "home_dyn_cluster_name",
    "away_dyn_cluster_name",
]

target_col = "home_win"

# Safety check: make sure all required columns exist
missing_numeric = [c for c in feature_cols_numeric if c not in matchups.columns]
missing_categorical = [c for c in feature_cols_categorical if c not in matchups.columns]

if missing_numeric or missing_categorical:
    raise KeyError(
        f"Missing numeric features: {missing_numeric}\n"
        f"Missing categorical features: {missing_categorical}"
    )

# Drop rows with missing values in features or target
model_df = matchups.dropna(
    subset=feature_cols_numeric + feature_cols_categorical + [target_col]
).reset_index(drop=True)

print("Modeling DF:", model_df.shape)

# Train / test split by season
train_mask = model_df["season"] <= 2023
test_mask = model_df["season"] == 2024

train_df = model_df[train_mask].copy()
test_df = model_df[test_mask].copy()

print("Train rows:", len(train_df))
print("Test rows :", len(test_df))

X_train = train_df[feature_cols_numeric + feature_cols_categorical]
y_train = train_df[target_col]

X_test = test_df[feature_cols_numeric + feature_cols_categorical]
y_test = test_df[target_col]

X_all = model_df[feature_cols_numeric + feature_cols_categorical]

print("Feature count:", X_train.shape[1])

Modeling DF: (1392, 71)
Train rows: 1107
Test rows : 285
Feature count: 11


## 4. Multi-model training & 2024 evaluation

In [5]:
# 3. Multi-model training and evaluation on 2024

# Preprocessing pipeline
numeric_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, feature_cols_numeric),
        ("cat", categorical_transformer, feature_cols_categorical),
    ]
)

# Base models
base_models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=4,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1,
    ),
    "GradientBoosting": GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        random_state=42,
    ),
}

if HAS_XGB:
    base_models["XGBoost"] = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=3,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1,
    )

base_models["MLP"] = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation="relu",
    solver="adam",
    max_iter=500,
    random_state=42,
)

results = []
model_probs_test = {}
model_probs_all = {}

print("Starting multi-model training...")

for name, estimator in base_models.items():
    print(f"\n=== {name} ===")
    clf = Pipeline(
        steps=[
            ("preprocess", preprocess),
            ("clf", estimator),
        ]
    )

    clf.fit(X_train, y_train)

    prob_test = clf.predict_proba(X_test)[:, 1]
    pred_test = (prob_test >= 0.5).astype(int)

    acc = accuracy_score(y_test, pred_test)
    brier = brier_score_loss(y_test, prob_test)
    auc = roc_auc_score(y_test, prob_test)

    print(f"{name}: acc={acc:.4f}, brier={brier:.4f}, auc={auc:.4f}")

    if name == "LogisticRegression":
        cm = confusion_matrix(y_test, pred_test)
        print("\nConfusion matrix (true rows, predicted cols):")
        print(cm)

    model_probs_test[name] = prob_test
    model_probs_all[name] = clf.predict_proba(X_all)[:, 1]

    results.append(
        {
            "model": name,
            "accuracy": acc,
            "brier": brier,
            "auc": auc,
        }
    )

results_df = pd.DataFrame(results)
results_df

Starting multi-model training...

=== LogisticRegression ===
LogisticRegression: acc=0.6316, brier=0.2258, auc=0.6745

Confusion matrix (true rows, predicted cols):
[[ 71  58]
 [ 47 109]]

=== RandomForest ===
RandomForest: acc=0.5965, brier=0.2314, auc=0.6514

=== GradientBoosting ===
GradientBoosting: acc=0.6140, brier=0.2405, auc=0.6326

=== XGBoost ===
XGBoost: acc=0.5789, brier=0.2527, auc=0.6036

=== MLP ===
MLP: acc=0.5509, brier=0.3203, auc=0.5828


/opt/homebrew/anaconda3/envs/ds_study/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,model,accuracy,brier,auc
0,LogisticRegression,0.631579,0.225832,0.674518
1,RandomForest,0.596491,0.231353,0.651411
2,GradientBoosting,0.614035,0.240524,0.632628
3,XGBoost,0.578947,0.252654,0.603608
4,MLP,0.550877,0.320263,0.582787


## 5. Choose simulation model & attach probability column

In [6]:
# 4. Attach per-model probabilities and choose one for simulations

# Add a column for each model's probability on all seasons
for name, arr in model_probs_all.items():
    col_name = f"pred_home_win_prob_{name}"
    model_df[col_name] = arr

print("Probability columns added:")
print([c for c in model_df.columns if c.startswith("pred_home_win_prob_")])

# Choose which model to use for simulations
SIM_MODEL_NAME = "LogisticRegression"  # change this if you want another model

sim_col = f"pred_home_win_prob_{SIM_MODEL_NAME}"
if sim_col not in model_df.columns:
    raise KeyError(
        f"Simulation column {sim_col} not found. "
        f"Available: {[c for c in model_df.columns if c.startswith('pred_home_win_prob_')]}"
    )

# Main probability column used downstream
model_df["pred_home_win_prob"] = model_df[sim_col]

print(f"Simulation will use model: {SIM_MODEL_NAME}")
print("model_df shape:", model_df.shape)

model_df[[
    "season", "week", "game_id",
    "home_team", "away_team", "home_win",
    "pred_home_win_prob"
]].head(10)

Probability columns added:
['pred_home_win_prob_LogisticRegression', 'pred_home_win_prob_RandomForest', 'pred_home_win_prob_GradientBoosting', 'pred_home_win_prob_XGBoost', 'pred_home_win_prob_MLP']
Simulation will use model: LogisticRegression
model_df shape: (1392, 77)


,season,week,game_id,home_team,away_team,home_win,pred_home_win_prob
0,2020,2,2020_02_WAS_ARI,ARI,WAS,1,0.597952
1,2020,3,2020_03_DET_ARI,ARI,DET,0,0.687061
2,2020,7,2020_07_SEA_ARI,ARI,SEA,1,0.368381
3,2020,9,2020_09_MIA_ARI,ARI,MIA,0,0.678243
4,2020,10,2020_10_BUF_ARI,ARI,BUF,1,0.483656
5,2020,13,2020_13_LA_ARI,ARI,LA,0,0.459642
6,2020,15,2020_15_PHI_ARI,ARI,PHI,1,0.524638
7,2020,16,2020_16_SF_ARI,ARI,SF,0,0.383087
8,2021,2,2021_02_MIN_ARI,ARI,MIN,1,0.505490
9,2021,5,2021_05_SF_ARI,ARI,SF,1,0.653928


### 6. 2024 Expected Wins from Pre-game Model

- Using the pre-game LogisticRegression model, I computed win probabilities for each 2024 game.
- For each team, expected wins are the sum of game-level win probabilities (home: P(home win), away: 1 − P(home win)).
- The 2024 power ranking by expected wins puts PHI, BUF, BAL, and KC in the top tier.

## 6. 2024 expected wins & toy Super Bowl probabilities

In [10]:
# 4. Attach per-model probabilities and choose one for simulations

# Add a column for each model's probability on all seasons
for name, arr in model_probs_all.items():
    col_name = f"pred_home_win_prob_{name}"
    model_df[col_name] = arr

print("Probability columns added:")
print([c for c in model_df.columns if c.startswith("pred_home_win_prob_")])

# Choose which model to use for simulations
SIM_MODEL_NAME = "LogisticRegression"  # change this if you want another model

sim_col = f"pred_home_win_prob_{SIM_MODEL_NAME}"
if sim_col not in model_df.columns:
    raise KeyError(
        f"Simulation column {sim_col} not found. "
        f"Available: {[c for c in model_df.columns if c.startswith('pred_home_win_prob_')]}"
    )

# Main probability column used downstream
model_df["pred_home_win_prob"] = model_df[sim_col]

print(f"Simulation will use model: {SIM_MODEL_NAME}")
print("model_df shape:", model_df.shape)

model_df[[
    "season", "week", "game_id",
    "home_team", "away_team", "home_win",
    "pred_home_win_prob"
]].head(10)



Probability columns added:
['pred_home_win_prob_LogisticRegression', 'pred_home_win_prob_RandomForest', 'pred_home_win_prob_GradientBoosting', 'pred_home_win_prob_XGBoost', 'pred_home_win_prob_MLP']
Simulation will use model: LogisticRegression
model_df shape: (1392, 77)


,season,week,game_id,home_team,away_team,home_win,pred_home_win_prob
0,2020,2,2020_02_WAS_ARI,ARI,WAS,1,0.597952
1,2020,3,2020_03_DET_ARI,ARI,DET,0,0.687061
2,2020,7,2020_07_SEA_ARI,ARI,SEA,1,0.368381
3,2020,9,2020_09_MIA_ARI,ARI,MIA,0,0.678243
4,2020,10,2020_10_BUF_ARI,ARI,BUF,1,0.483656
5,2020,13,2020_13_LA_ARI,ARI,LA,0,0.459642
6,2020,15,2020_15_PHI_ARI,ARI,PHI,1,0.524638
7,2020,16,2020_16_SF_ARI,ARI,SF,0,0.383087
8,2021,2,2021_02_MIN_ARI,ARI,MIN,1,0.505490
9,2021,5,2021_05_SF_ARI,ARI,SF,1,0.653928


In [12]:
# Cell 7 — 2024 expected wins per team

import numpy as np
import pandas as pd

# 1) Filter 2024 season games only
df_2024 = model_df[model_df["season"] == 2024].copy()

print(f"Number of 2024 games in model_df: {len(df_2024)}")

# 2) Expected win probability for each side
#    home_expected_win = P(home_win)
#    away_expected_win = 1 - P(home_win)
df_2024["home_exp_win"] = df_2024["pred_home_win_prob"]
df_2024["away_exp_win"] = 1.0 - df_2024["pred_home_win_prob"]

# 3) Aggregate to team-level
home_exp = (
    df_2024.groupby("home_team")["home_exp_win"]
    .sum()
    .rename("exp_wins_home")
)

away_exp = (
    df_2024.groupby("away_team")["away_exp_win"]
    .sum()
    .rename("exp_wins_away")
)

team_exp = (
    pd.concat([home_exp, away_exp], axis=1)
    .fillna(0.0)
)

team_exp["exp_wins_2024"] = (
    team_exp["exp_wins_home"] + team_exp["exp_wins_away"]
)

team_exp = (
    team_exp[["exp_wins_2024"]]
    .reset_index()
    .rename(columns={"index": "team"})
    .sort_values("exp_wins_2024", ascending=False)
    .reset_index(drop=True)
)

print("Top 10 expected wins for 2024:")
team_exp.head(10)

Number of 2024 games in model_df: 285
Top 10 expected wins for 2024:


,team,exp_wins_2024
0,PHI,13.469593
1,BUF,12.688104
2,BAL,12.442864
3,KC,11.594820
4,DET,10.801828
5,GB,10.794655
6,SF,10.685940
7,TB,10.004357
8,LA,9.616838
9,CIN,9.383046


### 7. Toy Super Bowl Probabilities from Expected Wins

- As a simple toy forecast, I convert 2024 expected wins into Super Bowl win probabilities using a softmax over `exp_wins_2024`.
- This does **not** model conferences, playoff brackets, or schedule; it only reflects relative team strength implied by the pre-game model.
- In this toy model, PHI emerges as the strongest favorite (~42%), followed by BUF and BAL in the next tier.

In [14]:
# Cell 7 — Toy Super Bowl probabilities from expected wins

alpha = 1.0  # temperature; >1 makes distribution sharper, <1 flatter

scores = team_exp["exp_wins_2024"].values
probs = np.exp(alpha * scores)
probs = probs / probs.sum()

sb_probs = team_exp.copy()
sb_probs["sb_win_prob"] = probs

sb_probs = sb_probs.sort_values("sb_win_prob", ascending=False).reset_index(drop=True)

print("Top 10 Super Bowl win probabilities (toy model):")
sb_probs.head(10)

Top 10 Super Bowl win probabilities (toy model):


,team,exp_wins_2024,sb_win_prob
0,PHI,13.469593,0.420348
1,BUF,12.688104,0.192403
2,BAL,12.442864,0.150559
3,KC,11.594820,0.064477
4,DET,10.801828,0.029175
5,GB,10.794655,0.028967
6,SF,10.685940,0.025983
7,TB,10.004357,0.013142
8,LA,9.616838,0.008920
9,CIN,9.383046,0.007061


### 8. 2024 Expected Wins vs Actual Record

- Using the 2024 games in `model_df`, I compute actual wins per team from the true game outcomes.
- Then I merge them with the model-based `exp_wins_2024` to measure team-level errors.
- I report MAE and RMSE across all teams, and list which teams were predicted most accurately and which were the biggest misses.

In [15]:
# Cell X — Compare 2024 expected wins vs actual record

import numpy as np
import pandas as pd

# 1) Keep only 2024 games from model_df
games_2024 = model_df[model_df["season"] == 2024].copy()

# (Optional) If you want regular season only, uncomment this:
# games_2024 = games_2024[games_2024["week"] <= 18].copy()

print("Number of 2024 games used:", len(games_2024))

# 2) Build actual wins per team
# Home side: home_win = 1 if home team won
home_results = games_2024[["home_team", "home_win"]].rename(
    columns={"home_team": "team", "home_win": "win"}
)

# Away side: away wins when home_win == 0
away_tmp = games_2024[["away_team", "home_win"]].copy()
away_tmp["win"] = 1 - away_tmp["home_win"]  # away win indicator
away_results = away_tmp[["away_team", "win"]].rename(
    columns={"away_team": "team"}
)

# Concatenate and aggregate
actual_wins_2024 = (
    pd.concat([home_results, away_results], axis=0)
    .groupby("team", as_index=False)["win"]
    .sum()
    .rename(columns={"win": "actual_wins_2024"})
)

print("Actual 2024 wins (sample):")
print(actual_wins_2024.head())

# 3) Merge with expected wins
# team_exp is assumed to have columns ["team", "exp_wins_2024"]
compare_df = team_exp.merge(actual_wins_2024, on="team", how="inner")

# 4) Compute errors
compare_df["error"] = compare_df["exp_wins_2024"] - compare_df["actual_wins_2024"]
compare_df["abs_error"] = compare_df["error"].abs()

# 5) Summary metrics
mae = compare_df["abs_error"].mean()
rmse = np.sqrt((compare_df["error"] ** 2).mean())

print("\n=== 2024 Expected vs Actual Wins ===")
print(f"Teams: {len(compare_df)}")
print(f"MAE (|exp - actual|): {mae:.3f} wins")
print(f"RMSE: {rmse:.3f} wins")

# 6) Show teams with smallest and largest errors
print("\nTop 10 most accurate teams (smallest abs error):")
print(
    compare_df.sort_values("abs_error")
    [["team", "exp_wins_2024", "actual_wins_2024", "error", "abs_error"]]
    .head(10)
)

print("\nTop 10 biggest misses (largest abs error):")
print(
    compare_df.sort_values("abs_error", ascending=False)
    [["team", "exp_wins_2024", "actual_wins_2024", "error", "abs_error"]]
    .head(10)
)

Number of 2024 games used: 285
Actual 2024 wins (sample):
  team  actual_wins_2024
0  ARI                 8
1  ATL                 8
2  BAL                13
3  BUF                15
4  CAR                 5

=== 2024 Expected vs Actual Wins ===
Teams: 32
MAE (|exp - actual|): 2.399 wins
RMSE: 2.941 wins

Top 10 most accurate teams (smallest abs error):
   team  exp_wins_2024  actual_wins_2024     error  abs_error
7    TB      10.004357                10  0.004357   0.004357
22  ATL       8.040958                 8  0.040958   0.040958
19  IND       8.202382                 8  0.202382   0.202382
5    GB      10.794655                11 -0.205345   0.205345
31  CAR       5.382269                 5  0.382269   0.382269
9   CIN       9.383046                 9  0.383046   0.383046
2   BAL      12.442864                13 -0.557136   0.557136
25  ARI       7.233688                 8 -0.766312   0.766312
10  MIA       9.324321                 8  1.324321   1.324321
16  DEN       8.640206  

### 8. 2024 Expected Wins vs Actual Record

Using the 2024 games in `model_df`, I compare the model-based expected wins (`exp_wins_2024`) with the actual win totals by team.

- MAE is about 2.4 wins per team and RMSE is about 2.9 wins.
- For several teams (e.g., TB, ATL, IND, GB), the model is within one win of the true record.
- The largest misses (e.g., NO, DET, TEN, LV) highlight limitations of a purely pre-game EPA and style–based model, where injuries, coaching changes, and unexpected breakout seasons are not fully captured.

## 9 . Prepare 2024 schedule with win probabilities

In [16]:
# Cell 8 — Prepare 2024 schedule with win probabilities

import numpy as np
import pandas as pd

# 1) Filter 2024 season from model_df
if "model_df" not in globals():
    raise RuntimeError("model_df is not defined. Run the previous cells first.")

season_2024 = model_df[model_df["season"] == 2024].copy()

required_cols = [
    "season",
    "week",
    "game_id",
    "home_team",
    "away_team",
    "pred_home_win_prob",
]

missing = [c for c in required_cols if c not in season_2024.columns]
if missing:
    raise KeyError(f"Missing columns in season_2024: {missing}")

season_2024 = season_2024[required_cols].reset_index(drop=True)

print("Number of 2024 games for simulation:", len(season_2024))
season_2024.head(10)

Number of 2024 games for simulation: 285


,season,week,game_id,home_team,away_team,pred_home_win_prob
0,2024,2,2024_02_LA_ARI,ARI,LA,0.428721
1,2024,3,2024_03_DET_ARI,ARI,DET,0.485045
2,2024,4,2024_04_WAS_ARI,ARI,WAS,0.542006
3,2024,7,2024_07_LAC_ARI,ARI,LAC,0.402783
4,2024,9,2024_09_CHI_ARI,ARI,CHI,0.354011
5,2024,10,2024_10_NYJ_ARI,ARI,NYJ,0.535376
6,2024,14,2024_14_SEA_ARI,ARI,SEA,0.507493
7,2024,15,2024_15_NE_ARI,ARI,NE,0.454399
8,2024,18,2024_18_SF_ARI,ARI,SF,0.351172
9,2024,1,2024_01_PIT_ATL,ATL,PIT,0.523213


## 10. Monte Carlo season simulation (2024 wins distribution)

In [17]:
# Cell 9 — Monte Carlo simulation for 2024 regular season wins

N_SIM = 10000  # number of season simulations

teams = sorted(
    set(season_2024["home_team"]).union(season_2024["away_team"])
)

num_games = len(season_2024)
num_teams = len(teams)

print("Teams:", teams)
print("Number of teams:", num_teams)
print("Number of games:", num_games)

# Matrix: simulations x teams
win_counts = np.zeros((N_SIM, num_teams), dtype=int)

p_home = season_2024["pred_home_win_prob"].values

for s in range(N_SIM):
    # 1) Draw random outcomes for each game
    u = np.random.rand(num_games)
    home_win_sim = (u < p_home).astype(int)
    away_win_sim = 1 - home_win_sim  # ignore ties for now

    # 2) Build per-game simulated result table
    sim_df = season_2024[["home_team", "away_team"]].copy()
    sim_df["home_win_sim"] = home_win_sim
    sim_df["away_win_sim"] = away_win_sim

    # 3) Aggregate wins by team in this simulation
    wins_by_team = pd.concat(
        [
            sim_df[["home_team", "home_win_sim"]].rename(
                columns={"home_team": "team", "home_win_sim": "win"}
            ),
            sim_df[["away_team", "away_win_sim"]].rename(
                columns={"away_team": "team", "away_win_sim": "win"}
            ),
        ],
        axis=0,
        ignore_index=True,
    ).groupby("team")["win"].sum()

    # 4) Align to full team list and store
    wins_by_team = wins_by_team.reindex(teams, fill_value=0)
    win_counts[s, :] = wins_by_team.values

# Build win distribution DataFrame
win_dist = pd.DataFrame(win_counts, columns=teams)

summary_2024 = pd.DataFrame(
    {
        "team": teams,
        "mean_wins": win_dist.mean(axis=0).values,
        "std_wins": win_dist.std(axis=0).values,
        "p_10plus": (win_dist >= 10).mean(axis=0).values,
        "p_5orless": (win_dist <= 5).mean(axis=0).values,
    }
).sort_values("mean_wins", ascending=False).reset_index(drop=True)

print("Top 10 teams by mean simulated wins:")
summary_2024.head(10)

Teams: ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS']
Number of teams: 32
Number of games: 285
Top 10 teams by mean simulated wins:


,team,mean_wins,std_wins,p_10plus,p_5orless
0,PHI,13.4590,2.112952,0.9662,0.0003
1,BUF,12.6822,2.093858,0.9337,0.0002
2,BAL,12.4609,2.008003,0.9298,0.0002
3,KC,11.5783,2.129536,0.8376,0.0014
4,GB,10.8181,2.024061,0.7470,0.0047
5,DET,10.7892,2.028885,0.7407,0.0050
6,SF,10.6659,1.940684,0.7325,0.0053
7,TB,9.9678,2.025086,0.5921,0.0141
8,LA,9.5896,2.149985,0.5116,0.0271
9,CIN,9.4049,1.980795,0.4856,0.0234


## 11. Compare MC mean wins vs analytic expected wins

In [18]:
# Cell 10 — Compare analytic expected wins vs Monte Carlo mean wins

if "team_exp" not in globals():
    raise RuntimeError("team_exp is not defined. Run the expected-wins cell first.")

compare = summary_2024.merge(
    team_exp[["team", "exp_wins_2024"]],
    on="team",
    how="left",
)

compare = compare.sort_values("exp_wins_2024", ascending=False).reset_index(drop=True)

compare[["team", "exp_wins_2024", "mean_wins", "std_wins", "p_10plus", "p_5orless"]].head(15)

,team,exp_wins_2024,mean_wins,std_wins,p_10plus,p_5orless
0,PHI,13.469593,13.4590,2.112952,0.9662,0.0003
1,BUF,12.688104,12.6822,2.093858,0.9337,0.0002
2,BAL,12.442864,12.4609,2.008003,0.9298,0.0002
3,KC,11.594820,11.5783,2.129536,0.8376,0.0014
4,DET,10.801828,10.7892,2.028885,0.7407,0.0050
5,GB,10.794655,10.8181,2.024061,0.7470,0.0047
6,SF,10.685940,10.6659,1.940684,0.7325,0.0053
7,TB,10.004357,9.9678,2.025086,0.5921,0.0141
8,LA,9.616838,9.5896,2.149985,0.5116,0.0271
9,CIN,9.383046,9.4049,1.980795,0.4856,0.0234


## 12. Toy Monte Carlo Super Bowl champion (from season results)

In [19]:
# Cell 7 — Toy Super Bowl probabilities from expected wins

alpha = 1.0  # temperature; >1 makes distribution sharper, <1 flatter

scores = team_exp["exp_wins_2024"].values
probs = np.exp(alpha * scores)
probs = probs / probs.sum()

sb_probs = team_exp.copy()
sb_probs["sb_win_prob"] = probs

sb_probs = sb_probs.sort_values("sb_win_prob", ascending=False).reset_index(drop=True)

print("Top 10 Super Bowl win probabilities (toy model):")
sb_probs.head(10)

Top 10 Super Bowl win probabilities (toy model):


,team,exp_wins_2024,sb_win_prob
0,PHI,13.469593,0.420348
1,BUF,12.688104,0.192403
2,BAL,12.442864,0.150559
3,KC,11.594820,0.064477
4,DET,10.801828,0.029175
5,GB,10.794655,0.028967
6,SF,10.685940,0.025983
7,TB,10.004357,0.013142
8,LA,9.616838,0.008920
9,CIN,9.383046,0.007061
